this is good shit: https://www.kaggle.com/imdevskp/covid-19-analysis-visualization-comparisons

this is gold: https://www.kaggle.com/gpreda/covid-19-romania-county-level

In [ ]:
!pip install plotly==4.14.3
!pip install unidecode

     |████████████████████████████████| 245kB 5.9MB/s 


In [ ]:
import requests
import os.path as path
import pandas as pd
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import folium
from unidecode import unidecode
from folium.plugins import HeatMap, HeatMapWithTime

In [ ]:
guvid = "guvid.xlsx"
vaccin = "vaccin.xlsx"
populatie ="excel-populatie.xls"

def remove_anthet(fileName):
  wb = openpyxl.load_workbook(fileName)
  for spreadsheetName in wb.sheetnames:
    sheet = wb[spreadsheetName]
    sheet.delete_rows(sheet.min_row, 2)
  wb.save(fileName)

url_stats = 'https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/d0b60b45-fb08-4980-a34c-8cbb4a43cad3/download/transparenta_martie_2021.xlsx'
url_vaccinuri = 'https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/bc19c354-644d-4a24-a26f-512129dbc70d/download/vaccinare-covid19-grupe-risc-01-25.03.2021.xlsx'
if(not(path.exists(guvid))):  
  r = requests.get(url_stats, allow_redirects=True)
  open(guvid, 'wb').write(r.content)
  remove_anthet(guvid)
else:
  print("guvid exists")

'''

if(not(path.exists(vaccin))):  
  r = requests.get(url_vaccinuri, allow_redirects=True)
  open(vaccin, 'wb').write(r.content)
else:
  print("vaccin exists")

'''

'\n\nif(not(path.exists(vaccin))):  \n  r = requests.get(url_vaccinuri, allow_redirects=True)\n  open(vaccin, \'wb\').write(r.content)\nelse:\n  print("vaccin exists")\n\n'

In [ ]:
df = px.data.stocks()
print(df)

           date      GOOG      AAPL      AMZN        FB      NFLX      MSFT
0    2018-01-01  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000
1    2018-01-08  1.018172  1.011943  1.061881  0.959968  1.053526  1.015988
2    2018-01-15  1.032008  1.019771  1.053240  0.970243  1.049860  1.020524
3    2018-01-22  1.066783  0.980057  1.140676  1.016858  1.307681  1.066561
4    2018-01-29  1.008773  0.917143  1.163374  1.018357  1.273537  1.040708
..          ...       ...       ...       ...       ...       ...       ...
100  2019-12-02  1.216280  1.546914  1.425061  1.075997  1.463641  1.720717
101  2019-12-09  1.222821  1.572286  1.432660  1.038855  1.421496  1.752239
102  2019-12-16  1.224418  1.596800  1.453455  1.104094  1.604362  1.784896
103  2019-12-23  1.226504  1.656000  1.521226  1.113728  1.567170  1.802472
104  2019-12-30  1.213014  1.678000  1.503360  1.098475  1.540883  1.788185

[105 rows x 7 columns]


In [ ]:
# Reading covid transparency dataset
excelDataCovid = pd.ExcelFile(guvid)
dfIncidenta = excelDataCovid.parse('incidenta')
dfTesteJudet = excelDataCovid.parse('teste_judet')
dfTesteRapideJudet = excelDataCovid.parse('teste_rapide_judet')
dfRataPozitivLaborator = excelDataCovid.parse('rata_pozitiv_laborator')
dfTestareLaborator = excelDataCovid.parse('testare_laborator')
dfDurataRecultareRezultat = excelDataCovid.parse('durata_recoltare_rezultat')
dfReinfectareJudet = excelDataCovid.parse('reinfectare_judet')
dfDecedateJudet = excelDataCovid.parse('decedati_judet')
dfPersoaneSpital = excelDataCovid.parse('persoane_spital')
dfPersoaneSpitalAti = excelDataCovid.parse('persoane_spital_ati')
dfPersoanteSpitalAtiJudet = excelDataCovid.parse('persoane_spital_ati_judet')
dfExternatiVindecatiSpital = excelDataCovid.parse('externati_vindecati_spital')
dfExternatiVindecatiJudet = excelDataCovid.parse('externati_vindecati_judet')
dfPopulatie = pd.read_excel(populatie)

In [ ]:
def diacritic_corect(nume):
    newString = ""
    for idx, litera in enumerate(nume):
      if(litera == 'î'):
        if(idx == 0 or( idx > 1 and nume[idx-1] == ' ')):
          newString += nume[idx]
        else:
          newString += 'â'
      else:
          newString += nume[idx]
    return newString

In [ ]:
population_dict = {
    'rimetea' : 1126,
    'saniob' : 2133,
    'sanmihaiudecampie' : 1459,
    'silivasudecampie' : 	872,
    'valenidambovita' : 2754,
    'varfuri' : 1842,
    'barca' : 3689,
    'gangiova' : 2478,
    'radulesti' : 1358,
    'acas' : 2827,
    'barghis' : 2015
}

def name_parse(name):
    new_name = name.lower()
    new_name = diacritic_corect(new_name)
    new_name = new_name.replace("-"," ")
    new_name = new_name.replace(" ","")
    return unidecode(new_name)

def add_population_dict(df):
  for index, row in df.iterrows():
    locality = row['LOCALITATE']
    population = row['POPULATIE']
    if(locality == "  ORAS INSURATEI"):
      print(name_parse(locality))
    population_dict[name_parse(locality)] = population

def convert_columns_to_rows(df):
  static = df.columns[0:2]  
  df1 = df.melt(id_vars=static, 
        var_name="data", 
        value_name="incidenta")
  return df1

def get_population(row):
  if(name_parse(row['UAT']) == "orasansuratei"):
    print(row['UAT'])
  return population_dict[name_parse(row['UAT'])]

def add_population_to_df(df):
  print(df)
  df['Populatie'] = df.apply (lambda row: get_population(row), axis=1)
  return df

In [ ]:
dfIncidentaWithDates = convert_columns_to_rows(dfIncidenta)
add_population_dict(dfPopulatie)
dfIncidentaWithDates = add_population_to_df(dfIncidentaWithDates)

orasinsuratei
               UAT                 Judet        data  incidenta
0            ALBAC                  ALBA  2021-03-01   0.504286
1      ALMAŞU MARE                  ALBA  2021-03-01   0.000000
2         ARIEŞENI                  ALBA  2021-03-01   0.000000
3      AVRAM IANCU                  ALBA  2021-03-01   0.000000
4          BERGHIN                  ALBA  2021-03-01   2.608242
...            ...                   ...         ...        ...
86017     SECTOR 2  MUNICIPIUL BUCUREŞTI  2021-03-27   7.137052
86018     SECTOR 3  MUNICIPIUL BUCUREŞTI  2021-03-27   5.865786
86019     SECTOR 4  MUNICIPIUL BUCUREŞTI  2021-03-27   7.113016
86020     SECTOR 5  MUNICIPIUL BUCUREŞTI  2021-03-27   6.374706
86021     SECTOR 6  MUNICIPIUL BUCUREŞTI  2021-03-27   7.488193

[86022 rows x 4 columns]


In [ ]:
dfIncidentaWithDates

,UAT,Judet,data,incidenta,Populatie
0,ALBAC,ALBA,2021-03-01,0.504286,2089
1,ALMAŞU MARE,ALBA,2021-03-01,0.000000,1289
2,ARIEŞENI,ALBA,2021-03-01,0.000000,1765
3,AVRAM IANCU,ALBA,2021-03-01,0.000000,3316
4,BERGHIN,ALBA,2021-03-01,2.608242,1893
...,...,...,...,...,...
86017,SECTOR 2,MUNICIPIUL BUCUREŞTI,2021-03-27,7.137052,345370
86018,SECTOR 3,MUNICIPIUL BUCUREŞTI,2021-03-27,5.865786,385439
86019,SECTOR 4,MUNICIPIUL BUCUREŞTI,2021-03-27,7.113016,287828
86020,SECTOR 5,MUNICIPIUL BUCUREŞTI,2021-03-27,6.374706,271575


In [ ]:
#add_population_to_df(dfIncidenta)
print(dfIncidenta.columns)
#print(population_dict)

Index(['UAT', 'Judet', 'data', 'incidenta'], dtype='object')


In [ ]:
# Reading vaccine transparency dataset
excelDataVaccine = pd.ExcelFile(vaccin)
vaccineDf = excelDataVaccine.parse('ag-grid')

In [ ]:

#grafic incidenta (bars), interactiv pe judet / comuna, sortare judete interactiv

In [ ]:
#total teste / judet, interactiv pe zi, combinat PCR + rapide, sortare judete interactiv

In [ ]:
#raport pozitivare / judet, interactiv pe zi, culori diferite in functie de valoare, sortare judete interactiv

In [ ]:
#decese / judet, interactiv pe zi, sortare judete interactiv

In [ ]:
#pers spitalizate / judet, interactiv pe zi (same + ATI)

In [ ]:
#extragere numar maxim de locuri per spital (same + ATI)

In [ ]:
#vindecari / judet sau spital / zi

In [ ]:
#grafic cu toate judetele, evolutia

In [ ]:
#rata mortalitatii

In [ ]:
#grafic cu decese, cazuri active, vindecari combinat

In [ ]:
#treemap judete

In [ ]:
#nr infectari / pop totala